## Using res

In [ ]:
import numpy as np
import os
import cv2


# ==============================================================================

# SCALE_TO_255
# ==============================================================================

def scale_to_255(a, min, max, dtype=np.uint8):
    """Scales an array of values from specified min, max range to 0-255."""
    return (((a - min) / float(max - min)) * 255).astype(dtype)


# ==============================================================================

# POINT_CLOUD_2_BIRDSEYE
# ==============================================================================

def point_cloud_2_birdseye(points,
                           res=0.1, # cell (voxel) size
                           side_range=(-30., 30.),  # left-most to right-most
                           fwd_range=(0., 60.),  # back-most to forward-most
                           height_range=(-2.73, 1.27),  # bottom-most to upper-most
                           color_by_distance=True):
    """Creates a 2D bird's eye view representation of the point cloud data."""
    # Extract x, y, z
    x_points = points[:, 0]
    y_points = points[:, 1]
    z_points = points[:, 2]

    # Apply filters for the region of interest
    f_filt = np.logical_and((x_points > fwd_range[0]), (x_points < fwd_range[1]))
    s_filt = np.logical_and((y_points > side_range[0]), (y_points < side_range[1]))
    filter = np.logical_and(f_filt, s_filt)
    indices = np.argwhere(filter).flatten()

    # Keep only the filtered points
    x_points = x_points[indices]
    y_points = y_points[indices]
    z_points = z_points[indices]

    # Compute distance for coloring
    if color_by_distance:
        distances = np.sqrt(x_points**2 + y_points**2)
        max_dist = np.sqrt(fwd_range[1]**2 + side_range[1]**2)
        colors = scale_to_255(distances, 0, max_dist)
    else:
        colors = scale_to_255(z_points, height_range[0], height_range[1])

    # Convert to pixel positions
    x_img = ((y_points - side_range[0]) / res).astype(np.int32) #  Lateral coordinates (Y-axis)
    y_img = ((x_points - fwd_range[0]) / res).astype(np.int32) # Forward-backward coordinates (X-axis)

    # Initialize the image array
    x_max = int((side_range[1] - side_range[0]) / res) # Width of image
    y_max = int((fwd_range[1] - fwd_range[0]) / res) # Height of image
    print(f'Width of BEV: {x_max}')
    print(f'Height of BEV: {y_max}')
    print(f'Cell resolution: {res}')
    print(f'x-range: {(fwd_range)}')
    print(f'y-range: {(side_range)}')
    print(f'z-range: {(height_range)}')
   
    # Mirroring the Y-axis (vertical, because OpenCV builds the image from top to bottom)
    y_img = y_max - y_img - 1
    # Mirroring of the X-axis (horizontal, to display the image correctly)
    x_img = x_max - x_img - 1

    im = np.zeros((y_max, x_max, 3), dtype=np.uint8)

    colors_bgr = cv2.applyColorMap(colors.reshape(-1, 1), cv2.COLORMAP_JET)  # Apply colormap to distances
    for i in range(len(x_img)):
        if 0 <= x_img[i] < x_max and 0 <= y_img[i] < y_max:
            im[y_img[i], x_img[i]] = colors_bgr[i, 0]  # Assign color

    return im


# ==============================================================================

# VISUALIZE_BEV_FROM_FOLDER
# ==============================================================================

def visualize_bev_from_file(bin_path, res=0.1, side_range=(-30., 30.), fwd_range=(0., 60.), height_range=(-2.73, 1.27)):
    """Visualizes a Bird's-Eye-View (BEV) from a single .bin file."""
    if not os.path.exists(bin_path):
        print(f"File not found: {bin_path}")
        return

    # Load point cloud data
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)  # Assumes x, y, z, intensity

    # Generate BEV image with coloring by distance
    bev_image = point_cloud_2_birdseye(points, res, side_range, fwd_range, height_range, color_by_distance=True)

    # Display with OpenCV
    cv2.imshow("Bird's Eye View", bev_image)
    while 1:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


# ==============================================================================

# MAIN FUNCTION
# ==============================================================================

if __name__ == "__main__":
    folder_path = "/home/rlab10/OpenPCDet/data/kitti/training/velodyne/000000.bin"
    #  KITTI default: [0, -40, -3, 70.4, 40, 1]
    # BEVDetnet: [0, -30, -2.73, 60, 30, 1.27]
    # for BEV 640x640 (YOLOv8 OBB) following parameters are needed: y(-32,32), x(0, 64)
    visualize_bev_from_file(folder_path, res=0.1, side_range=(-30., 30.), fwd_range=(0., 60.), height_range=(-2.73, 1.27))


Width of BEV: 600
Height of BEV: 600
Cell resolution: 0.1
x-range: (0.0, 60.0)
y-range: (-30.0, 30.0)
z-range: (-2.73, 1.27)


## Using width, height

In [17]:
import numpy as np
import os
import cv2


# ==============================================================================

# SCALE_TO_255
# ==============================================================================

def scale_to_255(a, min, max, dtype=np.uint8):
    """Scales an array of values from specified min, max range to 0-255."""
    return (((a - min) / float(max - min)) * 255).astype(dtype)


# ==============================================================================

# POINT_CLOUD_2_BIRDSEYE
# ==============================================================================

def point_cloud_2_birdseye(points,
                           width=640,  # Feste Breite des Bildes (in Pixel)
                           height=640,  # Feste Höhe des Bildes (in Pixel)
                           side_range=(-30., 30.),  # left-most to right-most
                           fwd_range=(0., 60.),  # back-most to forward-most
                           height_range=(-2.73, 1.27),  # bottom-most to upper-most
                           color_by_distance=True):
    """Creates a 2D bird's eye view representation of the point cloud data."""
    
    # Berechnung der Auflösung auf Basis der gewünschten Bildgröße
    res_x = (side_range[1] - side_range[0]) / width
    res_y = (fwd_range[1] - fwd_range[0]) / height
    #res = (res_x + res_y) / 2  # Mittelwert der Auflösungen für x und y
    res = 0.1

    # Extract x, y, z
    x_points = points[:, 0]
    y_points = points[:, 1]
    z_points = points[:, 2]

    # Apply filters for the region of interest
    f_filt = np.logical_and((x_points > fwd_range[0]), (x_points < fwd_range[1]))
    s_filt = np.logical_and((y_points > side_range[0]), (y_points < side_range[1]))
    filter = np.logical_and(f_filt, s_filt)
    indices = np.argwhere(filter).flatten()

    # Keep only the filtered points
    x_points = x_points[indices]
    y_points = y_points[indices]
    z_points = z_points[indices]
   

    # Compute distance for coloring
    if color_by_distance:
        distances = np.sqrt(x_points**2 + y_points**2)
        max_dist = np.sqrt(fwd_range[1]**2 + side_range[1]**2)
        colors = scale_to_255(distances, 0, max_dist)
    else:
        colors = scale_to_255(z_points, height_range[0], height_range[1])

    # Convert to pixel positions
    x_img = ((y_points - side_range[0]) / res).astype(np.int32)
    y_img = ((x_points - fwd_range[0]) / res).astype(np.int32)
    y_img = height - y_img - 1
    x_img = width - x_img -1
    print(f'Width of BEV: {width}')
    print(f'Height of BEV: {height}')
    print(f'Cell resolution: {res}')
    print(f'x-range: {(fwd_range)}')
    print(f'y-range: {(side_range)}')
    print(f'z-range: {(height_range)}')

    # Initialize the image array with the desired dimensions
    im = np.zeros((height, width, 3), dtype=np.uint8)

    # Apply color mapping
    colors_bgr = cv2.applyColorMap(colors.reshape(-1, 1), cv2.COLORMAP_JET)  # Apply colormap to distances
    for i in range(len(x_img)):
        if 0 <= x_img[i] < width and 0 <= y_img[i] < height:
            im[y_img[i], x_img[i]] = colors_bgr[i, 0]  # Assign color

    return im

# ==============================================================================

# VISUALIZE_BEV_FROM_FILE
# ==============================================================================

def visualize_bev_from_file(bin_path, width=640, height=640, side_range=(-30., 30.), fwd_range=(0., 60.), height_range=(-2.73, 1.27)):
    """Visualizes a Bird's-Eye-View (BEV) from a single .bin file."""
    if not os.path.exists(bin_path):
        print(f"File not found: {bin_path}")
        return

    # Load point cloud data
    points = np.fromfile(bin_path, dtype=np.float32).reshape(-1, 4)  # Assumes x, y, z, intensity

    # Generate BEV image with coloring by distance
    bev_image = point_cloud_2_birdseye(points, width, height, side_range, fwd_range, height_range, color_by_distance=True)

    # Display with OpenCV
    cv2.imshow("Bird's Eye View", bev_image)
    while 1:
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cv2.destroyAllWindows()


# ==============================================================================

# MAIN FUNCTION
# ==============================================================================

if __name__ == "__main__":
    folder_path = "/home/rlab10/OpenPCDet/data/kitti/training/velodyne/000001.bin"
    visualize_bev_from_file(folder_path, width=640, height=640, side_range=(-30., 30.), fwd_range=(0., 60), height_range=(-2.73, 1.27))


Width of BEV: 640
Height of BEV: 640
Cell resolution: 0.1
x-range: (0.0, 60)
y-range: (-30.0, 30.0)
z-range: (-2.73, 1.27)
